In [1]:

import uuid
import pandas as pd
import sys
from pprint import pprint


import sys, os
sys.path.append(os.path.abspath('.'))

from itertools import *
import ccxt

import ccxt.pro as ccxtpro
from asyncio import run
import asyncio
from threading import Thread

from typing import Literal

In [2]:
from api_config import *
# side = Literal['buy', 'sell']
buy = 'buy'
sell = 'sell'
creds


{'apiKey': '655f763effa2720001d2f3f4',
 'secret': '1f52656e-dd97-4c72-9a8b-eb0dc3171f8d',
 'password': '#2M%RSN$c*4f1jDYVCiOUmTJf$#RYF'}

In [3]:

print('python', sys.version)
print('CCXT Version:', ccxt.__version__)


python 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
CCXT Version: 4.1.63


In [4]:

exchange = ccxt.kucoinfutures({
    'adjustForTimeDifference': True,
    **creds
})
# exchange.verbose = True

securities = pd.DataFrame(exchange.load_markets()).transpose()
securities.head()
# pprint(securities)

,id,lowercaseId,symbol,base,quote,settle,baseId,quoteId,settleId,type,...,expiryDatetime,strike,optionType,precision,limits,created,info,percentage,tierBased,tiers
BTC/USDT:USDT,XBTUSDTM,None,BTC/USDT:USDT,BTC,USDT,USDT,XBT,USDT,USDT,swap,...,None,None,None,"{'amount': 1.0, 'price': 0.1, 'cost': None, 'b...","{'leverage': {'min': 1.0, 'max': 125.0}, 'amou...",1585555200000,"{'symbol': 'XBTUSDTM', 'rootSymbol': 'USDT', '...",True,True,"{'taker': [[0.0, 0.0006], [50.0, 0.0006], [200..."
BTC/USD:BTC,XBTUSDM,None,BTC/USD:BTC,BTC,USD,BTC,XBT,USD,XBT,swap,...,None,None,None,"{'amount': 1.0, 'price': 0.1, 'cost': None, 'b...","{'leverage': {'min': 1.0, 'max': 50.0}, 'amoun...",1552638575000,"{'symbol': 'XBTUSDM', 'rootSymbol': 'XBT', 'ty...",True,True,"{'taker': [[0.0, 0.0006], [50.0, 0.0006], [200..."
ETH/USDT:USDT,ETHUSDTM,None,ETH/USDT:USDT,ETH,USDT,USDT,ETH,USDT,USDT,swap,...,None,None,None,"{'amount': 1.0, 'price': 0.01, 'cost': None, '...","{'leverage': {'min': 1.0, 'max': 100.0}, 'amou...",1591086000000,"{'symbol': 'ETHUSDTM', 'rootSymbol': 'USDT', '...",True,True,"{'taker': [[0.0, 0.0006], [50.0, 0.0006], [200..."
BCH/USDT:USDT,BCHUSDTM,None,BCH/USDT:USDT,BCH,USDT,USDT,BCH,USDT,USDT,swap,...,None,None,None,"{'amount': 1.0, 'price': 0.01, 'cost': None, '...","{'leverage': {'min': 1.0, 'max': 75.0}, 'amoun...",1595406600000,"{'symbol': 'BCHUSDTM', 'rootSymbol': 'USDT', '...",True,True,"{'taker': [[0.0, 0.0006], [50.0, 0.0006], [200..."
BSV/USDT:USDT,BSVUSDTM,None,BSV/USDT:USDT,BSV,USDT,USDT,BSV,USDT,USDT,swap,...,None,None,None,"{'amount': 1.0, 'price': 0.01, 'cost': None, '...","{'leverage': {'min': 1.0, 'max': 50.0}, 'amoun...",1595406600000,"{'symbol': 'BSVUSDTM', 'rootSymbol': 'USDT', '...",True,True,"{'taker': [[0.0, 0.0006], [50.0, 0.0006], [200..."


In [88]:
# x: side = sidel

In [5]:

class asset:
    def __init__(self, exchange, symbol, risk, leverage):
        self.exchange = exchange
        self.symbol = symbol
        self.risk = risk
        self.leverage = leverage


    def bracket_open(self, price, side, down_price, up_price,  type='market', sl_type='market', tp_type='market'):
        sl_price, tp_price = (up_price, down_price) if side == sell else (down_price, up_price)
        return self.open(price, side, sl_price, tp_price,  type='market', sl_type='market', tp_type='market')

    def open(self, price, side, sl_price, tp_price,  type='market', sl_type='market', tp_type='market'):
        if price == 'last':
            price = self.exchange.fetch_ticker(self.symbol)['last']
        direction = 1 if side == buy else -1 
        size = self.risk / (price - sl_price) / self.leverage * direction

        params = {
            'leverage': self.leverage,
            # "stopLossPrice": sl_price,
            # "stopPriceType":  ('down' if side == 'buy' else 'up'),
            # 'triggerPriceType': 'last'
        }
        print('Placing order:', self.symbol,  type, side, size, price)
        self.order = self.exchange.createOrder(self.symbol, type, side, size, price=price, params=params)

        sl_params = {
            'leverage': self.leverage,
            "stopLossPrice": sl_price,
            # "stopPriceType":  ('down' if side == 'buy' else 'up'),
            'triggerPriceType': 'last',
            'reduceOnly': True
        }
        self.sl_order = self.exchange.createOrder(self.symbol, sl_type, ('sell' if side == 'buy' else 'buy'), size, price=sl_price, params=sl_params)

        tp_params = {
            'leverage': self.leverage,
            'takeProfitPrice': tp_price,
            # "stopPriceType":  ('down' if side == 'buy' else 'up'),
            'triggerPriceType': 'last',
            'reduceOnly': True
        }
        self.tp_order = self.exchange.createOrder(self.symbol, tp_type, ('sell' if side == 'buy' else 'buy'), size, price=price, params=tp_params)

    def close(self, type='market', price=None):
        tkr = self.exchange.fetch_ticker(self.symbol)
        if exchange.futuresprivate_get_position({'symbol': self.symbol})['data']['isOpen']:
            self.exchange.futuresPrivatePostOrders({
                    "clientOid": "aaa",
                    "price": tkr["last"], 
                    "type":'market',
                    "reduceOnly": True,
                    "closeOrder": True,
                    "symbol": self.symbol,
            })    
        for ord in exchange.futuresprivate_get_orders({'symbol': 'ALGOUSDTM', 'status': 'active'})['data']['items']:
            exchange.cancel_order(ord['id'])    

        exchange.futuresprivate_delete_stoporders({'symbol': self.symbol})

    
        

In [6]:
algo = asset(exchange, 'ALGOUSDTM', 10, 10)

In [76]:
algo.bracket_open(0.1348, sell, 0.1338, 0.1353, type='limit' )
# algo.order, algo.sl_order, algo.tp_order

Placing order: ALGOUSDTM market sell 3999.9999999999964 0.1348


In [73]:
algo.close()

In [30]:
eth = asset(exchange, 'ETHUSDTM', 25, 1)

In [31]:
eth.bracket_open(2028, buy, 2007, 2068, type='limit', tp_type='limit' )

Placing order: ETHUSDTM market buy 1.1904761904761905 2028


In [13]:
eth.close()

In [ ]:
# exchange.futuresprivate_get_position({'symbol': 'ALGOUSDTM'})
# exchange.cancel({'symbol': 'ALGOUSDTM'})
    # exchna
exchange.futuresprivate_get_orders({'symbol': 'ALGOUSDTM', 'status': 'active'})

In [ ]:
algo.close()

In [24]:
leverage = 1
risk = 25
symbol = 'ETHUSDTM'
sl_price = 1971
type, side, price = 'limit', buy, 1990
size = risk / (price - sl_price) / leverage 
params = {
    'leverage': leverage,
}
print('Placing order:', symbol,  type, side, size, price)
order = exchange.create_order(symbol, type, side, size, price=price, params=params)
order

Placing order: ETHUSDTM limit buy 1.3157894736842106 1990


{'id': '119445795538792448',
 'clientOrderId': None,
 'timestamp': None,
 'datetime': None,
 'lastTradeTimestamp': None,
 'symbol': None,
 'type': None,
 'side': None,
 'price': None,
 'amount': None,
 'cost': None,
 'average': None,
 'filled': None,
 'remaining': None,
 'status': None,
 'fee': None,
 'trades': [],
 'timeInForce': None,
 'postOnly': None,
 'stopPrice': None,
 'triggerPrice': None,
 'info': {'code': '200000', 'data': {'orderId': '119445795538792448'}},
 'fees': [],
 'lastUpdateTimestamp': None,
 'reduceOnly': None,
 'takeProfitPrice': None,
 'stopLossPrice': None}

In [ ]:
stop_ws_orders = True
# orders_thread.join()

In [ ]:
stop_orders_thread = False
async def ws_orders(stop):
    exchange = ccxtpro.kucoinfutures({'newUpdates': False})
    while True:
        orderbook = await exchange.watch_order_book('ADAUSDTM')
        print(orderbook['asks'][0], orderbook['bids'][0])
        if stop(): break
    await exchange.close()


orders_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

orders_thread = Thread(target=setup_loop, args=(orders_loop,))
orders_thread.start()
orders_future = asyncio.run_coroutine_threadsafe(ws_orders(lambda: stop_orders_thread), orders_loop)  

# run(main())